# XOR Multiple Inputs/Targets

Testing...

In [1]:
from conx import Network, Layer, SGD

Using Theano backend.


In [2]:
net = Network("XOR", 2, 5, 2, activation="sigmoid")
net.connect()
dataset = [
    ([0, 0], [0, 0]),
    ([0, 1], [1, 1]),
    ([1, 0], [1, 1]),
    ([1, 1], [0, 0])
]

In [3]:
net.set_dataset(dataset)

Split dataset into:
   4 train inputs
   0 test inputs
Input Summary:
   length  : 4
   training: 4
   testing : 0
   shape  : (2,)
   range  : (0.0, 1.0)
Target Summary:
   length  : 4
   training: 4
   testing : 0
   shape  : (2,)
   range  : (0.0, 1.0)


In [4]:
net.get_target(0)

[0.0, 0.0]

In [5]:
net.compile(loss='mean_squared_error',
            optimizer=SGD(lr=0.3, momentum=0.9))

In [6]:
net.train(epochs=2000, accuracy=1.0, report_rate=25)

Training...
Epoch #   25 | train loss 0.25125 | train acc 0.75000 | validate% 0.00000
Epoch #   50 | train loss 0.25007 | train acc 0.50000 | validate% 0.00000
Epoch #   75 | train loss 0.24970 | train acc 0.50000 | validate% 0.00000
Epoch #  100 | train loss 0.24896 | train acc 0.50000 | validate% 0.00000
Epoch #  125 | train loss 0.24674 | train acc 0.50000 | validate% 0.00000
Epoch #  150 | train loss 0.23926 | train acc 0.50000 | validate% 0.00000
Epoch #  175 | train loss 0.21957 | train acc 0.50000 | validate% 0.00000
Epoch #  200 | train loss 0.18405 | train acc 0.25000 | validate% 0.00000
Epoch #  225 | train loss 0.14289 | train acc 0.50000 | validate% 0.00000
Epoch #  250 | train loss 0.09708 | train acc 0.50000 | validate% 0.00000
Epoch #  275 | train loss 0.05578 | train acc 0.50000 | validate% 0.00000
Epoch #  300 | train loss 0.03226 | train acc 0.50000 | validate% 0.00000
Epoch #  325 | train loss 0.02081 | train acc 0.50000 | validate% 0.25000
Epoch #  350 | train loss 

In [7]:
net.test()

Testing...
# | inputs | targets | outputs
0 | [0.0, 0.0] | [0.0, 0.0] | [0.0, 0.0]
1 | [0.0, 1.0] | [1.0, 1.0] | [0.9, 0.9]
2 | [1.0, 0.0] | [1.0, 1.0] | [0.9, 0.9]
3 | [1.0, 1.0] | [0.0, 0.0] | [0.1, 0.1]


In [8]:
net.make_widget()

<IPython.core.display.Javascript object>

In [9]:
net.propagate_to("input", [0, 1])

[0.0, 1.0]

In [10]:
net.propagate([0.5, 0.5])

[0.21613617, 0.23198915]

In [11]:
net.propagate_to("hidden", [1, 0])

[0.000480635033454746,
 0.014374571852385998,
 0.8942601084709167,
 0.5025742650032043,
 0.40186434984207153]

In [12]:
net.propagate_to("output", [0.25, 0.75])

[0.5994237661361694, 0.6120522618293762]

In [13]:
net.propagate_to("input", [0.25, 0.25])

[0.25, 0.25]

In [14]:
net.propagate_from("input", [1.0, 1.0])

[0.67862564, 0.64391273]

In [15]:
net.propagate_from("hidden", [1.0, 1.0, 0.0, 0.0, 0.0])

[0.7737425, 0.68368661]

In [16]:
net.test()

Testing...
# | inputs | targets | outputs
0 | [0.0, 0.0] | [0.0, 0.0] | [0.0, 0.0]
1 | [0.0, 1.0] | [1.0, 1.0] | [0.9, 0.9]
2 | [1.0, 0.0] | [1.0, 1.0] | [0.9, 0.9]
3 | [1.0, 1.0] | [0.0, 0.0] | [0.1, 0.1]


In [1]:
from conx import Network, Layer, SGD

Using Theano backend.


In [2]:
net = Network("XOR2")
net.add(Layer("input1", shape=1))
net.add(Layer("input2", shape=1))
net.add(Layer("hidden1", shape=10, activation="sigmoid"))
net.add(Layer("hidden2", shape=10, activation="sigmoid"))
net.add(Layer("shared-hidden", shape=5, activation="sigmoid"))
net.add(Layer("output1", shape=1, activation="sigmoid"))
net.add(Layer("output2", shape=1, activation="sigmoid"))

In [3]:
net

<Network name='XOR2'>

In [4]:
net.connect("input1", "hidden1")
net.connect("input2", "hidden2")
net.connect("hidden1", "shared-hidden")
net.connect("hidden2", "shared-hidden")
net.connect("shared-hidden", "output1")
net.connect("shared-hidden", "output2")

In [5]:
net.set_input_layer_order("input1", "input2")
net.set_output_layer_order("output1", "output2")

In [6]:
net.compile(loss='mean_squared_error',
            optimizer=SGD(lr=0.3, momentum=0.9))

In [7]:
dataset = [
    ([[0],[0]], [[0],[0]]),
    ([[0],[1]], [[1],[1]]),
    ([[1],[0]], [[1],[1]]),
    ([[1],[1]], [[0],[0]])
]

In [8]:
net.set_dataset(dataset)

Split dataset into:
   4 train inputs
   0 test inputs
Input Summary:
   length  : 4
   training: 4
   testing : 0
   shape  : [(1,), (1,)]
   range  : (0.0, 1.0)
Target Summary:
   length  : 4
   training: 4
   testing : 0
   shape  : [(1,), (1,)]
   range  : (0.0, 1.0)


In [9]:
net.make_widget()

<IPython.core.display.Javascript object>

In [10]:
net["shared-hidden"].colormap = "spring"
net.propagate_from("hidden1", [0.0, 1.0] * 5)

[[0.32813856], [0.91043729]]

In [11]:
net["shared-hidden"].colormap = "hot"
net.propagate_from("hidden1", [1.0, 1.0] * 5, ["shared-hidden"])

[0.48424909, 0.32282391, 0.69763553, 0.22763956, 0.146277]

In [12]:
net.propagate_to("output1", [[1], [0]])

[0.34739741683006287]

In [13]:
for i in range(20):
    (epoch_count, loss, acc, val_acc) = net.train(epochs=100, verbose=0)
    for index in range(4):
        net.propagate(dataset[index][0])

In [13]:
net.train(epochs=2000, accuracy=1.0, report_rate=25)

Training...
Epoch #   25 | train loss 0.50041 | train acc 1.00000 | validate% 0.00000
Epoch #   50 | train loss 0.50024 | train acc 1.00000 | validate% 0.00000
Epoch #   75 | train loss 0.50011 | train acc 1.00000 | validate% 0.00000
Epoch #  100 | train loss 0.50007 | train acc 1.00000 | validate% 0.00000
Epoch #  125 | train loss 0.50004 | train acc 1.00000 | validate% 0.00000
Epoch #  150 | train loss 0.50001 | train acc 1.00000 | validate% 0.00000
Epoch #  175 | train loss 0.49998 | train acc 1.00000 | validate% 0.00000
Epoch #  200 | train loss 0.49995 | train acc 1.00000 | validate% 0.00000
Epoch #  225 | train loss 0.49992 | train acc 1.00000 | validate% 0.00000
Epoch #  250 | train loss 0.49988 | train acc 1.00000 | validate% 0.00000
Epoch #  275 | train loss 0.49983 | train acc 1.00000 | validate% 0.00000
Epoch #  300 | train loss 0.49976 | train acc 1.00000 | validate% 0.00000
Epoch #  325 | train loss 0.49967 | train acc 1.00000 | validate% 0.00000
Epoch #  350 | train loss 

In [14]:
net.propagate_from("input1", [0.0])

[[0.5], [0.5]]

In [15]:
net.propagate_from("shared-hidden", [0.0] * 5)

[[0.5], [0.5]]

In [16]:
net.propagate_to("hidden1", [[1], [1]])

[0.792599081993103,
 0.782215416431427,
 0.84234219789505,
 0.1320749670267105,
 0.1610153168439865,
 0.1744602918624878,
 0.4586453139781952,
 0.1609877198934555,
 0.3683990240097046,
 0.15291506052017212]

In [17]:
net.test()

Testing...
# | inputs | targets | outputs
0 | [[0.0], [0.0]] | [[0.0], [0.0]] | [[0.1], [0.1]]
1 | [[0.0], [1.0]] | [[1.0], [1.0]] | [[0.9], [0.9]]
2 | [[1.0], [0.0]] | [[1.0], [1.0]] | [[0.9], [0.9]]
3 | [[1.0], [1.0]] | [[0.0], [0.0]] | [[0.1], [0.1]]


In [18]:
net.slice_dataset(2)

Slicing dataset 0:2...
Input Summary:
   length  : 2
   training: 2
   testing : 0
   shape  : [(1,), (1,)]
   range  : (0.0, 1.0)
Target Summary:
   length  : 2
   training: 2
   testing : 0
   shape  : [(1,), (1,)]
   range  : (0.0, 1.0)


In [19]:
net.train(epochs=2000, accuracy=1.0, report_rate=25)

Training...
Epoch #  687 | train loss 0.01742 | train acc 2.00000 | validate% 1.00000


# Conx model is a Keras Model

In [34]:
from keras.utils.vis_utils import model_to_dot
from IPython.display import HTML

In [35]:
dot = model_to_dot(net.model)

In [36]:
HTML(dot.create_svg().decode())